# 16. Modelos de regresion con algoritmos de Machine Learning
***


### Caso de estudio #1



### Tecnicas de ajuste de los datos y modelos:

| Ingeniería de datos       | Si / No       |
| :-------                  | :------:    |
| Escalado de datos         | Si          |
| Sobremuestreo  | No        |
| Incluir variable ciclo | Si        |


In [105]:
# librerias
import numpy as np
import math
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import seaborn as sns

from numpy.random import seed
seed(0)
from sklearn.preprocessing import MinMaxScaler

from sklearn import linear_model
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
from numpy import mean
from numpy import std
from xgboost import XGBRegressor
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.neural_network import MLPRegressor



In [47]:
# funcion para calcular metricas de desempeño

def regression_metrics(y_test, y_pred2):
    # Mean Deviation (MD)
    MD=np.mean(y_test - y_pred2)
    # mean absolute error
    MAE=mean_absolute_error(y_test,y_pred2)
    # Mean Squared Error (MSE)
    MSE=mean_squared_error(y_test,y_pred2)
    # Root Mean Squared Error (RMSE)
    RMSE= sqrt(mean_squared_error(y_test,y_pred2))
    # Mean Percent Error (MPE)
    MPE=np.mean((y_test - y_pred2) / y_test) * 100
    # Mean Absolute Percent Error (MAPE)
    MAPE=np.mean(np.abs((y_test - y_pred2) / y_test)) * 100
    print('\n')
    print("METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION")
    print('-'*60)
    print(f'Mean Deviation (MD): {round(MD,2)}') 
    print(f'Mean Absolute Error (MAE): {round(MAE,2)}')
    print('-'*60)
    print(f'Mean Squared Error (MSE): {round(MSE,2)}') 
    print(f'Root Mean Squared Error (RMSE): {round(RMSE,2)}')
    print('-'*60)    
    print(f'Mean Percent Error (MPE): {round(MPE,2)}') 
    print(f'Mean Absolute Percent Error (MAPE): {round(MAPE,2)}')
    print('-'*60)


### Preparar datos

In [48]:
columns=["id","ciclo","set1","set2","set3","sensor1","sensor2","sensor3","sensor4","sensor5","sensor6","sensor7","sensor8",
         "sensor9","sensor10","sensor11","sensor12","sensor13","sensor14","sensor15","sensor16","sensor17","sensor18","sensor19"
         ,"sensor20","sensor21","sensor22","sensor23"]
train=pd.read_csv("train_FD001.txt",sep=" ",names=columns)
test=pd.read_csv("test_FD001.txt",sep=" ",names=columns)
rul=pd.read_csv("RUL_FD001.txt",sep=" ",header=None)
train.drop(['sensor22','sensor23'], axis=1, inplace=True)
test.drop(['sensor22','sensor23'], axis=1, inplace=True)
rul.drop([1], axis=1, inplace=True)
rul.columns = ['ttf']
rul2=pd.DataFrame(train.groupby('id')['ciclo'].max()).reset_index()
rul2.columns = ['id', 'falla']
train= train.merge(rul2, on=['id'], how='left')
train['ttf'] = train['falla'] - train['ciclo']
train.drop('falla', axis=1, inplace=True)
train.head()


,id,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,ttf
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [49]:
rul3=pd.DataFrame(test.groupby('id')['ciclo'].max()).reset_index()
rul3.columns = ['id', 'falla']
rul['id'] = rul.index + 1
rul['falla'] = rul3['falla'] + rul['ttf']
rul.drop('ttf', axis=1, inplace=True)
test=test.merge(rul, on=['id'], how='left')
test['ttf'] = test['falla'] - test['ciclo']
test.drop('falla', axis=1, inplace=True)
test.head()


,id,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,ttf
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,142
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,141
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,140
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,139
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,138


In [50]:
variables=['set1','set2','set3','sensor1','sensor2','sensor3','sensor4','sensor5','sensor6','sensor7','sensor8','sensor9','sensor10','sensor11','sensor12','sensor13','sensor14','sensor15','sensor16','sensor17','sensor18','sensor19','sensor20','sensor21']

objetivo='ttf'

In [51]:
# normalizado de datos sin incluir id, ciclo, ttf 
scaler = MinMaxScaler()
train[variables]=scaler.fit_transform(train[variables])
test[variables]=scaler.transform(test[variables])

In [52]:
train.head()

,id,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,ttf
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,191
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,190
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,189
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,188
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,187


In [53]:
test.head()

,id,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,...,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21,ttf
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,142
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,141
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,140
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,139
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,138


In [54]:
print("numero de dimensiones =", train.ndim)
print("tamaño de la tabla =", train.shape)
print("total datos =", train.size)

numero de dimensiones = 2
tamaño de la tabla = (20631, 27)
total datos = 557037


In [55]:
print("numero de dimensiones =", test.ndim)
print("tamaño de la tabla =", test.shape)
print("total datos =", test.size)

numero de dimensiones = 2
tamaño de la tabla = (13096, 27)
total datos = 353592


In [56]:
x_train=train.iloc[:,np.r_[1:26]]
x_train.head()


,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,...,0.633262,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662
1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,...,0.765458,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014
2,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,...,0.795309,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375
3,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,...,0.889126,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386
4,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,...,0.746269,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502


In [57]:
x_test=test.iloc[:,np.r_[1:26]]
x_test.head()

,ciclo,set1,set2,set3,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,...,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor18,sensor19,sensor20,sensor21
0,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,1.0,...,0.646055,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834
1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,1.0,...,0.739872,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827
2,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,1.0,...,0.699360,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348
3,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,1.0,...,0.573561,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110
4,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,1.0,...,0.737740,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377


In [58]:
y_train=train.iloc[:,26]
y_train.head()

0    191
1    190
2    189
3    188
4    187
Name: ttf, dtype: int64

In [59]:
y_test=test.iloc[:,26]
y_test.head()

0    142
1    141
2    140
3    139
4    138
Name: ttf, dtype: int64

# Linear Regression

In [60]:
modelo1=linear_model.LinearRegression()
modelo1.fit(x_train,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [61]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[182.3970959  188.1565842  174.59104337 ...  35.28571796  32.52945016
  16.323466  ]


In [62]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 3.64
Mean Absolute Error (MAE): 33.11
------------------------------------------------------------
Mean Squared Error (MSE): 1840.82
Root Mean Squared Error (RMSE): 42.9
------------------------------------------------------------
Mean Percent Error (MPE): -9.75
Mean Absolute Percent Error (MAPE): 29.21
------------------------------------------------------------


# Decision Tree Regressor

In [63]:
modelo1 = tree.DecisionTreeRegressor(random_state=0)
modelo1.fit(x_train,y_train)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=0, splitter='best')

In [64]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[176. 237. 160. ...  20.  33.  12.]


In [65]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 1.29
Mean Absolute Error (MAE): 45.28
------------------------------------------------------------
Mean Squared Error (MSE): 3587.86
Root Mean Squared Error (RMSE): 59.9
------------------------------------------------------------
Mean Percent Error (MPE): -6.93
Mean Absolute Percent Error (MAPE): 35.26
------------------------------------------------------------


# Random Forest Regressor

In [66]:
modelo1 =RandomForestRegressor(n_estimators=100, random_state=0)
modelo1.fit(x_train,y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [67]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[216.54 221.66 195.24 ...  25.16  22.56  16.1 ]


In [68]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 1.04
Mean Absolute Error (MAE): 31.68
------------------------------------------------------------
Mean Squared Error (MSE): 1720.82
Root Mean Squared Error (RMSE): 41.48
------------------------------------------------------------
Mean Percent Error (MPE): -7.51
Mean Absolute Percent Error (MAPE): 25.06
------------------------------------------------------------


# Support Vector Regression

In [69]:
modelo1 =SVR()
modelo1.fit(x_train,y_train)


C:\Users\alexh\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [70]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[178.20849349 187.23668932 191.91458317 ...  32.6060055   32.82645126
  30.9693715 ]


In [71]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 19.02
Mean Absolute Error (MAE): 35.32
------------------------------------------------------------
Mean Squared Error (MSE): 2304.95
Root Mean Squared Error (RMSE): 48.01
------------------------------------------------------------
Mean Percent Error (MPE): 4.71
Mean Absolute Percent Error (MAPE): 27.14
------------------------------------------------------------


# Gradient Boosting Regressor

In [72]:
modelo1 =ensemble.GradientBoostingRegressor(random_state=0)
modelo1.fit(x_train,y_train)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto', random_state=0,
                          subsample=1.0, tol=0.0001, validation_fraction=0.1,
                          verbose=0, warm_start=False)

In [73]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[209.25281057 214.68795789 202.45054803 ...  34.22455372  27.87892182
  20.12650354]


In [74]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 1.91
Mean Absolute Error (MAE): 30.87
------------------------------------------------------------
Mean Squared Error (MSE): 1664.68
Root Mean Squared Error (RMSE): 40.8
------------------------------------------------------------
Mean Percent Error (MPE): -6.8
Mean Absolute Percent Error (MAPE): 24.38
------------------------------------------------------------


# Bagging Regressor

In [75]:
modelo1 =ensemble.BaggingRegressor(random_state=0)
modelo1.fit(x_train,y_train)


BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=0, verbose=0,
                 warm_start=False)

In [76]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[259.9 197.  208.9 ...  23.9  24.2  16.3]


In [77]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 1.36
Mean Absolute Error (MAE): 33.28
------------------------------------------------------------
Mean Squared Error (MSE): 1897.03
Root Mean Squared Error (RMSE): 43.55
------------------------------------------------------------
Mean Percent Error (MPE): -7.2
Mean Absolute Percent Error (MAPE): 26.24
------------------------------------------------------------


# Logistic Regression

In [78]:
modelo1 =LogisticRegression(random_state=0)
modelo1.fit(x_train,y_train)


C:\Users\alexh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\alexh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [79]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[175 189 179 ...  23  33  24]


In [80]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 19.16
Mean Absolute Error (MAE): 35.98
------------------------------------------------------------
Mean Squared Error (MSE): 2329.03
Root Mean Squared Error (RMSE): 48.26
------------------------------------------------------------
Mean Percent Error (MPE): 6.31
Mean Absolute Percent Error (MAPE): 26.54
------------------------------------------------------------


# Extra-trees regressor

In [81]:
modelo1 =ensemble.ExtraTreesRegressor(random_state=0)
modelo1.fit(x_train,y_train)


C:\Users\alexh\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
                    max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                    oob_score=False, random_state=0, verbose=0,
                    warm_start=False)

In [82]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[194.6 229.8 200.  ...  24.   24.9  18.2]


In [83]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 1.05
Mean Absolute Error (MAE): 33.35
------------------------------------------------------------
Mean Squared Error (MSE): 1901.34
Root Mean Squared Error (RMSE): 43.6
------------------------------------------------------------
Mean Percent Error (MPE): -7.58
Mean Absolute Percent Error (MAPE): 26.24
------------------------------------------------------------


# Bayesian Ridge Regression

In [84]:
modelo1 =linear_model.BayesianRidge()
modelo1.fit(x_train,y_train)


BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
              fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
              normalize=False, tol=0.001, verbose=False)

In [85]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[182.3458058  188.20401057 174.71925662 ...  35.45755131  32.6971056
  16.58263547]


In [86]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 3.66
Mean Absolute Error (MAE): 33.12
------------------------------------------------------------
Mean Squared Error (MSE): 1840.89
Root Mean Squared Error (RMSE): 42.91
------------------------------------------------------------
Mean Percent Error (MPE): -9.74
Mean Absolute Percent Error (MAPE): 29.22
------------------------------------------------------------


# Passive Aggressive Regressor

In [90]:

modelo1 =PassiveAggressiveRegressor()
modelo1.fit(x_train,y_train)


PassiveAggressiveRegressor(C=1.0, average=False, early_stopping=False,
                           epsilon=0.1, fit_intercept=True,
                           loss='epsilon_insensitive', max_iter=1000,
                           n_iter_no_change=5, random_state=None, shuffle=True,
                           tol=0.001, validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [91]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[198.23077442 212.32029413 204.6072277  ... -41.92291489 -39.09684028
 -48.5193575 ]


In [92]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 24.23
Mean Absolute Error (MAE): 40.32
------------------------------------------------------------
Mean Squared Error (MSE): 3033.6
Root Mean Squared Error (RMSE): 55.08
------------------------------------------------------------
Mean Percent Error (MPE): 20.68
Mean Absolute Percent Error (MAPE): 36.19
------------------------------------------------------------


# AdaBoost regressor

In [93]:

modelo1 =AdaBoostRegressor(random_state=0)
modelo1.fit(x_train,y_train)


AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=0)

In [94]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[211.15059927 232.35774865 211.15059927 ...  76.79663394  58.34351145
  52.20980926]


In [95]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): -13.23
Mean Absolute Error (MAE): 36.7
------------------------------------------------------------
Mean Squared Error (MSE): 1996.87
Root Mean Squared Error (RMSE): 44.69
------------------------------------------------------------
Mean Percent Error (MPE): -20.29
Mean Absolute Percent Error (MAPE): 32.42
------------------------------------------------------------


# ElasticNet

In [99]:
modelo1 =ElasticNet(random_state=0)
modelo1.fit(x_train,y_train)


ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=0, selection='cyclic', tol=0.0001, warm_start=False)

In [100]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[186.14821626 187.08304709 184.60154495 ...  44.81329323  43.88220675
  41.39676949]


In [101]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 9.21
Mean Absolute Error (MAE): 34.76
------------------------------------------------------------
Mean Squared Error (MSE): 2040.57
Root Mean Squared Error (RMSE): 45.17
------------------------------------------------------------
Mean Percent Error (MPE): -7.57
Mean Absolute Percent Error (MAPE): 32.25
------------------------------------------------------------


# MLPRegressor

In [102]:

modelo1 =MLPRegressor(random_state=0)
modelo1.fit(x_train,y_train)


C:\Users\alexh\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=0, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [103]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[205.36204624 212.88253528 209.75090689 ...  31.19603308  31.68517238
  20.53715922]


In [104]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 2.45
Mean Absolute Error (MAE): 31.06
------------------------------------------------------------
Mean Squared Error (MSE): 1680.36
Root Mean Squared Error (RMSE): 40.99
------------------------------------------------------------
Mean Percent Error (MPE): -6.47
Mean Absolute Percent Error (MAPE): 24.56
------------------------------------------------------------


Teniendo en cuenta los resultados del error en lo modelos, vale la pena resaltar que los métodos de conjunto obtienen un buen rendimiento en este experimento, ya que si comparamos el algoritmo Gradient Boosting Regressor (GBR) con la red BLSTM, la diferencia en términos del MAPE es de tan solo 1.27%, analizando el RMSE, el algoritmo GBR alcanza el menor error del grupo de experimentos con un valor de 40.80 ciclos, con respecto al sesgo el MPE registra un -6.8\%, este valor negativo indica que los valores estimados por el modelo en general son mayores que los valores reales observadores.

# Explorar y ajustar hiperparametros de los algoritmos con menor error

## MLP grid search CV

In [28]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(random_state=0)

parameters= {'learning_rate':["constant", "invscaling", "adaptive"],
             'hidden_layer_sizes':[(10), (100)],
             'activation': ["relu"],
             'solver': ["lbfgs", "sgd", "adam"],
             'max_iter':[400]
}

acc_scorer=make_scorer(accuracy_score)

grid = GridSearchCV(model, param_grid=parameters, cv=2, verbose=2)
%time grid.fit(x_train, y_train)


Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV] activation=relu, hidden_layer_sizes=10, learning_rate=constant, max_iter=400, solver=lbfgs 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  activation=relu, hidden_layer_sizes=10, learning_rate=constant, max_iter=400, solver=lbfgs, total=   0.6s
[CV] activation=relu, hidden_layer_sizes=10, learning_rate=constant, max_iter=400, solver=lbfgs 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  activation=relu, hidden_layer_sizes=10, learning_rate=constant, max_iter=400, solver=lbfgs, total=   0.7s
[CV] activation=relu, hidden_layer_sizes=10, learning_rate=constant, max_iter=400, solver=sgd 


...

C:\Users\alexh\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  activation=relu, hidden_layer_sizes=10, learning_rate=adaptive, max_iter=400, solver=adam, total=   6.5s
[CV] activation=relu, hidden_layer_sizes=100, learning_rate=constant, max_iter=400, solver=lbfgs 
[CV]  activation=relu, hidden_layer_sizes=100, learning_rate=constant, max_iter=400, solver=lbfgs, total=   7.7s
[CV] activation=relu, hidden_layer_sizes=100, learning_rate=constant, max_iter=400, solver=lbfgs 
[CV]  activation=relu, hidden_layer_sizes=100, learning_rate=constant, max_iter=400, solver=lbfgs, total=   7.2s
[CV] activation=relu, hidden_layer_sizes=100, learning_rate=constant, max_iter=400, solver=sgd 
[CV]  activation=relu, hidden_layer_sizes=100, learning_rate=constant, max_iter=400, solver=sgd, total=   0.3s
[CV] activation=relu, hidden_layer_sizes=100, learning_rate=constant, max_iter=400, solver=sgd 
[CV]  activation=relu, hidden_layer_sizes=100, learning_rate=constant, max_iter=400, solver=sgd, total=   0.5s
[CV] activation=relu, hidden_layer_sizes=100, learnin

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  2.9min finished


Wall time: 3min 1s


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                    batch_size='auto', beta_1=0.9, beta_2=0.999,
                                    early_stopping=False, epsilon=1e-08,
                                    hidden_layer_sizes=(100,),
                                    learning_rate='constant',
                                    learning_rate_init=0.001, max_iter=200,
                                    momentum=0.9, n_iter_no_change=10,
                                    nesterovs_momentum=True, power_t=0.5,
                                    random_stat...er='adam',
                                    tol=0.0001, validation_fraction=0.1,
                                    verbose=False, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'activation': ['relu'],
                         'hidden_layer_sizes': [10, 100],
                         'lear

In [29]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_index_)

{'activation': 'relu', 'hidden_layer_sizes': 10, 'learning_rate': 'constant', 'max_iter': 400, 'solver': 'adam'}
0.6653521404820173
2


In [30]:
modelo1 =grid.best_estimator_
modelo1.fit(x_train,y_train)

# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)


[203.81144477 210.98274936 210.72427854 ...  39.51639281  41.25615351
  21.32175003]


In [31]:
# Root Mean Squared Error (RMSE)

rmsm2= sqrt(mean_squared_error(y_test,y_pred2))
print(rmsm2)

41.65633572241199


In [32]:
# mean absolute error

mean_absolute_error(y_test,y_pred2)

31.69811713674044

In [33]:
# Mean Absolute Percent Error (MAPE)

np.mean(np.abs((y_test - y_pred2) / y_test)) * 100

25.71377234140045

## Modelo ajustado MLP

In [106]:

modelo1 =MLPRegressor(random_state=0,activation='relu', hidden_layer_sizes=100,learning_rate='constant',max_iter=500, solver='adam') 
                                    
modelo1.fit(x_train,y_train)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=100, learning_rate='constant',
             learning_rate_init=0.001, max_iter=500, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=0, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [107]:
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[206.80497001 213.62815129 209.28926849 ...  31.71000624  31.63422899
  21.41987818]


In [108]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 3.65
Mean Absolute Error (MAE): 30.83
------------------------------------------------------------
Mean Squared Error (MSE): 1682.03
Root Mean Squared Error (RMSE): 41.01
------------------------------------------------------------
Mean Percent Error (MPE): -5.15
Mean Absolute Percent Error (MAPE): 24.0
------------------------------------------------------------


### nota:

Vale la pena resaltar que los metodos de conjunto obtienen un buen rendimiento en este experimento, especificamente el algoritmo Gradient Boosting Regressor (GBR), Por esta razon a continuacion se exploran
diferentes ajustes de hiperparametros e implementaciones eficientes con el GBR

# Ajuste de hiperparametros e implementaciones eficientes con el GBR

## Histogram-Based Gradient Boosting Machine for Regression repeated k-fold cross-validation

In [44]:
model = HistGradientBoostingRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
n_scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -25.226 (0.646)


In [112]:
model = HistGradientBoostingRegressor()
model.fit(x_train, y_train)

HistGradientBoostingRegressor(l2_regularization=0.0, learning_rate=0.1,
                              loss='least_squares', max_bins=256,
                              max_depth=None, max_iter=100, max_leaf_nodes=31,
                              min_samples_leaf=20, n_iter_no_change=None,
                              random_state=None, scoring=None, tol=1e-07,
                              validation_fraction=0.1, verbose=0)

In [113]:
# base a predecir con test
# calcular el error en la base de prueba

y_pred2=model.predict(x_test)
print(y_pred2)

[214.10768219 218.93341881 201.32051671 ...  30.67669868  25.16768806
  17.59427118]


In [114]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 1.86
Mean Absolute Error (MAE): 31.33
------------------------------------------------------------
Mean Squared Error (MSE): 1707.87
Root Mean Squared Error (RMSE): 41.33
------------------------------------------------------------
Mean Percent Error (MPE): -6.65
Mean Absolute Percent Error (MAPE): 24.58
------------------------------------------------------------


## Gradient Boosting With XGBoost k-fold CV

In [52]:
# modelo
model = XGBRegressor(objective='reg:squarederror')
cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
n_scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# entrenar el modelo
model = XGBRegressor(objective='reg:squarederror')
model.fit(x_train, y_train)

MAE: -25.418 (0.642)


C:\Users\alexh\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\alexh\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [115]:
# entrenar el modelo
model = XGBRegressor(objective='reg:squarederror')
model.fit(x_train, y_train)

C:\Users\alexh\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\alexh\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [116]:
# base a predecir con test
# calcular el error en la base de prueba

y_pred2=model.predict(x_test)
print(y_pred2)

[206.97037  215.28616  207.52292  ...  35.986664  29.019949  21.794836]


In [117]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 1.9
Mean Absolute Error (MAE): 30.87
------------------------------------------------------------
Mean Squared Error (MSE): 1661.94
Root Mean Squared Error (RMSE): 40.77
------------------------------------------------------------
Mean Percent Error (MPE): -6.8
Mean Absolute Percent Error (MAPE): 24.39
------------------------------------------------------------


## Gradient Boosting With LightGBM k-fold CV

In [62]:

model = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
n_scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# entrenar el modelo
model = LGBMRegressor()
model.fit(x_train, y_train)

MAE: -25.216 (0.652)


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [118]:
# entrenar el modelo
model = LGBMRegressor()
model.fit(x_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [119]:
# base a predecir con test
# calcular el error en la base de prueba

y_pred2=model.predict(x_test)
print(y_pred2)

[201.21470439 229.69151927 197.16847401 ...  33.67795333  28.11175017
  17.10707379]


In [121]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 2.02
Mean Absolute Error (MAE): 31.32
------------------------------------------------------------
Mean Squared Error (MSE): 1711.68
Root Mean Squared Error (RMSE): 41.37
------------------------------------------------------------
Mean Percent Error (MPE): -6.55
Mean Absolute Percent Error (MAPE): 24.56
------------------------------------------------------------


## Gradient Boosting Regressor Grid Search CV

In [17]:

model = ensemble.GradientBoostingRegressor(random_state=0)
parameters = {  "learning_rate": [0.01, 0.025, 0.1],
                "max_depth":[3,5,8],
                "max_features":["log2","sqrt"],
                "criterion": ["friedman_mse"],
                "subsample":[0.5, 0.8, 1.0],
                "n_estimators":[10, 100, 200 ]
            }

acc_scorer=make_scorer(accuracy_score)

grid = GridSearchCV(model, param_grid=parameters, cv=5, verbose=2)
%time grid.fit(x_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5 
[CV]  criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5 
[CV]  criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5 
[CV]  criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5 
[CV]  criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.8 
[CV]  criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.8, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.01, max_depth=3, max_features=log2, n_estimators=10, subsample=0.8 
[CV]  criterion=friedman_mse, learni

[Parallel(n_jobs=1)]: Done 810 out of 810 | elapsed: 12.4min finished


Wall time: 12min 23s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_iter...
                             

parameters = {  "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
                "min_samples_split": np.linspace(0.1, 0.5, 12),
                "min_samples_leaf": np.linspace(0.1, 0.5, 12),
                "max_depth":[3,5,8],
                "max_features":["log2","sqrt"],
                "criterion": ["friedman_mse",  "mae"],
                "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
                "n_estimators":[10]
            }

In [18]:
# mejores hiperparametros
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_index_)

{'criterion': 'friedman_mse', 'learning_rate': 0.025, 'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 200, 'subsample': 1.0}
0.7085670429786326
71


In [122]:
modelo1 =ensemble.GradientBoostingRegressor(random_state=0, n_estimators=200, criterion='friedman_mse', learning_rate=0.025, max_depth=3, max_features='sqrt', subsample=1.0)
modelo1.fit(x_train,y_train)
# base a predecir con test
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)


[198.00691324 203.77286705 197.05746671 ...  44.9868301   34.84132596
  21.54702571]


In [123]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 2.49
Mean Absolute Error (MAE): 30.31
------------------------------------------------------------
Mean Squared Error (MSE): 1625.2
Root Mean Squared Error (RMSE): 40.31
------------------------------------------------------------
Mean Percent Error (MPE): -7.13
Mean Absolute Percent Error (MAPE): 24.33
------------------------------------------------------------


## Grid Search CV Gradient Boosting With XGBoost

In [18]:
from sklearn.metrics import make_scorer, accuracy_score

model = XGBRegressor(objective='reg:squarederror')
parameters = {  "learning_rate": [0.01, 0.07, 0.1],
                "max_depth":[3,6,9],
                "min_child_weight":[2,4,5],
                "silent": [1],
                "subsample": [0.5,0.6],
                "colsample_bytree": [0.7,0.8,0.85],
                "n_estimators":[100, 500, 1000]
            }

acc_scorer=make_scorer(accuracy_score)

grid = GridSearchCV(model, param_grid=parameters, cv=3, verbose=2)
%time grid.fit(x_train, y_train)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits
[CV] colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=100, silent=1, subsample=0.5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=100, silent=1, subsample=0.5, total=   3.0s
[CV] colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=100, silent=1, subsample=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.9s remaining:    0.0s
C:\Users\alexh\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=100, silent=1, subsample=0.5, total=   3.4s
[CV] colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=100, silent=1, subsample=0.5 


...

C:\Users\alexh\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  colsample_bytree=0.85, learning_rate=0.1, max_depth=9, min_child_weight=5, n_estimators=1000, silent=1, subsample=0.6, total=  41.6s


[Parallel(n_jobs=1)]: Done 1458 out of 1458 | elapsed: 329.0min finished
C:\Users\alexh\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\alexh\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Wall time: 5h 29min 10s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:squarederror',
                                    random_st...
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'colsample_bytree': [0.7, 0.8, 0.85],
                         'learning_rate': [0.01, 0.07, 0.1],
                      

parameters = {  "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
                "min_samples_split": np.linspace(0.1, 0.5, 12),
                "min_samples_leaf": np.linspace(0.1, 0.5, 12),
                "max_depth":[3,5,8],
                "max_features":["log2","sqrt"],
                "criterion": ["friedman_mse",  "mae"],
                "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
                "n_estimators":[10]
            }

In [19]:
# mejores parametros
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_index_)

{'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 500, 'silent': 1, 'subsample': 0.5}
0.7052351144638258
2


In [125]:
modelo1 = XGBRegressor(colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=2, n_estimators=500, silent=1, subsample=0.5, objective='reg:squarederror')
modelo1.fit(x_train,y_train)
# base a predecir con test
# calcular el error en la base de prueba

y_pred2=modelo1.predict(x_test)
print(y_pred2)

[204.26099  209.31845  201.7408   ...  40.4878    33.64237   23.536907]


In [126]:
regression_metrics(y_test, y_pred2)



METRICAS DE DESEMPEÑO DEL MODELO DE REGRESION
------------------------------------------------------------
Mean Deviation (MD): 3.08
Mean Absolute Error (MAE): 30.41
------------------------------------------------------------
Mean Squared Error (MSE): 1640.74
Root Mean Squared Error (RMSE): 40.51
------------------------------------------------------------
Mean Percent Error (MPE): -6.13
Mean Absolute Percent Error (MAPE): 23.99
------------------------------------------------------------


El algoritmo GBR con sistema de refuerzo XGBoost (Extreme Gradient Boosting) obtiene los mejores resultados del grupo de los algoritmos de ML.